In [1]:
import spacy
import pandas as pd
import numpy as np

In [2]:
from collections import Counter
from string import punctuation

In [3]:
nlp = spacy.load("en_core_web_lg")

In [5]:
music_words= ['album', 'genre', 'music', 'albums', 'song', 'songs', 'genres', 'year', 'minutes', 'minute', 'years']
def get_hotwords(text, x, title, artist, entities):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        # 4
        if(token.text == title or token.text in artist.replace(',', '').split()):
            continue
            
        if(token.text in music_words):
            continue
        if(token.text in entities):
            continue
            
        if(token.pos_ in pos_tag):
            result.append(token.text)
            
    out = Counter(result).most_common(x)
                
    return out

In [9]:
data = pd.read_json("../solrData/clean_reviews.json", orient='records')
data

,reviewid,title,artist,score,author,review_publication_date,genre,song_release_year,record_label,review_content,keywords
0,22703,mezzanine,massive attack,9.3,nate patrin,1483833600000,electronic,1998,virgin,"“Trip-hop” eventually became a ’90s punchline,...","[late, del, naja, hop, band, 3d, andy, man, tr..."
1,22721,prelapsarian,krallice,7.9,zoe camp,1483747200000,metal,2016,hathenter,"Eight years, five albums, and two EPs in, the ...","[band, barr, metal, post, mcmaster, winter, de..."
2,22659,all of them naturals,uranium club,7.3,david glickman,1483747200000,rock,2016,"static shock,fashionable idiots",Minneapolis’ Uranium Club seem to revel in bei...,"[punk, band, human, exploration, ep, scene, na..."
3,22661,first songs,"kleenex, liliput",9.0,jenn pelly,1483660800000,rock,2016,"kill rock stars,mississippi",Kleenex began with a crash. It transpired one ...,"[punk, marder, scene, beri, early, chords, dru..."
4,22725,new start,taso,8.1,kevin lozano,1483660800000,electronic,2016,teklife,It is impossible to consider a given release b...,"[footwork, double, cup, rashad, way, start, tr..."
...,...,...,...,...,...,...,...,...,...,...,...
18412,1535,let us replay!,coldcut,8.9,james p. wisdom,917308800000,"jazz,electronic",0,ninja tune,The marketing guys of yer average modern megac...,"[brand, equity, pale, new, money, product, goo..."
18413,1341,1999,cassius,4.8,james p. wisdom,917308800000,electronic,0,astralwerks,"Well, it's been two weeks now, and I guess it'...","[techno, sure, club, weeks, time, guys, dilige..."
18414,5376,out of tune,mojave 3,6.3,jason josephes,916099200000,rock,0,4ad,"Out of Tune is a Steve Martin album. Yes, I'l...","[martin, man, slowdive, steve, funny, laughter..."
18415,2413,"singles breaking up, vol. 1",don caballero,7.2,james p. wisdom,916099200000,"rock,metal,experimental",1999,touch and go,"Well, kids, I just went back and re-read my re...","[sore-, jamming, review, sharp, drums, oppress..."


In [20]:
content = list(data['review_content'])
titles = list(data['title'])
artist = list(data['artist'])
key_words = []
entities = []
print(len(content))
for i in range(len(content)):
    print (i, end="\r")
    doc = nlp(content[i])
    ents = []
    entitiesss= []
    for ent in doc.ents:
        if ent.label != "DATE" or ent.label != "ORDINAL":
            entitiesss.append((ent.text, ent.label))
            text = ent.text.lower()
            text = text.replace('’', '')
            text = text.split()
            ents += text
    result = get_hotwords(content[i], 25, titles[i], artist[i], ents)
    key_words.append(result)
    entities.append(entitiesss)


18417


In [21]:
data['key_words_with_entities'] = key_words
data['entities'] = entities
data

,reviewid,title,artist,score,author,review_publication_date,genre,song_release_year,record_label,review_content,keywords,key_words_with_entities,entities
0,22703,mezzanine,massive attack,9.3,nate patrin,1483833600000,electronic,1998,virgin,"“Trip-hop” eventually became a ’90s punchline,...","[late, del, naja, hop, band, 3d, andy, man, tr...","[(late, 5), (hop, 4), (band, 4), (trip, 3), (s...","[(’90s, 391), (today, 391), (Bristol, 384), (m..."
1,22721,prelapsarian,krallice,7.9,zoe camp,1483747200000,metal,2016,hathenter,"Eight years, five albums, and two EPs in, the ...","[band, barr, metal, post, mcmaster, winter, de...","[(band, 4), (metal, 2), (post, 2), (dense, 2),...","[(Eight years, 391), (five, 397), (two, 397), ..."
2,22659,all of them naturals,uranium club,7.3,david glickman,1483747200000,rock,2016,"static shock,fashionable idiots",Minneapolis’ Uranium Club seem to revel in bei...,"[punk, band, human, exploration, ep, scene, na...","[(punk, 7), (band, 5), (ep, 2), (scene, 2), (s...","[(Minneapolis, 384), (Uranium Club, 383), (las..."
3,22661,first songs,"kleenex, liliput",9.0,jenn pelly,1483660800000,rock,2016,"kill rock stars,mississippi",Kleenex began with a crash. It transpired one ...,"[punk, marder, scene, beri, early, chords, dru...","[(punk, 9), (scene, 4), (beri, 3), (early, 3),...","[(Kleenex, 386), (one night, 392), (Zurich, 38..."
4,22725,new start,taso,8.1,kevin lozano,1483660800000,electronic,2016,teklife,It is impossible to consider a given release b...,"[footwork, double, cup, rashad, way, start, tr...","[(footwork, 11), (way, 4), (producer, 3), (bes...","[(DJ Rashad’s, 380), (Double Cup, 388), (four ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18412,1535,let us replay!,coldcut,8.9,james p. wisdom,917308800000,"jazz,electronic",0,ninja tune,The marketing guys of yer average modern megac...,"[brand, equity, pale, new, money, product, goo...","[(brand, 7), (equity, 7), (pale, 2), (new, 2),...","[(these days, 391), (six\n months, 391), (C..."
18413,1341,1999,cassius,4.8,james p. wisdom,917308800000,electronic,0,astralwerks,"Well, it's been two weeks now, and I guess it'...","[techno, sure, club, weeks, time, guys, dilige...","[(techno, 3), (sure, 3), (time, 1), (guys, 1),...","[(two weeks, 391), (Cassius, 380), ('70s, 391)..."
18414,5376,out of tune,mojave 3,6.3,jason josephes,916099200000,rock,0,4ad,"Out of Tune is a Steve Martin album. Yes, I'l...","[martin, man, slowdive, steve, funny, laughter...","[(funny, 4), (laughter, 3), (tune, 2), (time, ...","[(Steve Martin, 380), (Steve Martin, 380), (Ra..."
18415,2413,"singles breaking up, vol. 1",don caballero,7.2,james p. wisdom,916099200000,"rock,metal,experimental",1999,touch and go,"Well, kids, I just went back and re-read my re...","[sore-, jamming, review, sharp, drums, oppress...","[(sore-, 4), (jamming, 4), (review, 3), (sharp...","[(Don Caballero, 380), (NPR, 383), (This Ameri..."


In [22]:
data.to_json('reviews_with_keywordsEntities.json', orient='records')

In [4]:
data = pd.read_json('reviews_with_entities_split.json', orient='records')
data

,reviewid,title,artist,score,author,review_publication_date,genre,song_release_year,record_label,review_content,keywords,key_words_with_entities,entities,persons,places,facilities,work_of_art,organizations
0,22703,mezzanine,massive attack,9.3,nate patrin,1483833600000,electronic,1998,virgin,"“Trip-hop” eventually became a ’90s punchline,...","[late, del, naja, hop, band, 3d, andy, man, tr...","[[late, 5], [hop, 4], [band, 4], [trip, 3], [s...","[[’90s, 391], [today, 391], [Bristol, 384], [m...","[Portishead, Geoff Barrow, Portishead, Robert ...","[Bristol, Jamaica, Bristol, Memphis, UK, Algiers]",[Del Naja],"[The Conversation’s Gene Hackman, Inertia Cree...","[Pre-Millennium Tension, Massive Attack, Mezza..."
1,22721,prelapsarian,krallice,7.9,zoe camp,1483747200000,metal,2016,hathenter,"Eight years, five albums, and two EPs in, the ...","[band, barr, metal, post, mcmaster, winter, de...","[[band, 4], [metal, 2], [post, 2], [dense, 2],...","[[Eight years, 391], [five, 397], [two, 397], ...","[Colin Marston, Mick Barr’s, Lev Weinstein’s, ...","[New York, Prelapsarian, Prelapsarian]",[the Towers of Terror],"[Transformation Chronicles, Lotus Throne, Hate...","[Krallice, Krallice, releases—2015, Hyperion, ..."
2,22659,all of them naturals,uranium club,7.3,david glickman,1483747200000,rock,2016,"static shock,fashionable idiots",Minneapolis’ Uranium Club seem to revel in bei...,"[punk, band, human, exploration, ep, scene, na...","[[punk, 7], [band, 5], [ep, 2], [scene, 2], [s...","[[Minneapolis, 384], [Uranium Club, 383], [las...","[Devo, teddy]","[Minneapolis, Detroit]",[ ],"[Operation Pt, The Lottery, Opus, That Clown’s...","[Uranium Club, Human Exploration, Them Natural..."
3,22661,first songs,"kleenex, liliput",9.0,jenn pelly,1483660800000,rock,2016,"kill rock stars,mississippi",Kleenex began with a crash. It transpired one ...,"[punk, marder, scene, beri, early, chords, dru...","[[punk, 9], [scene, 4], [beri, 3], [early, 3],...","[[Kleenex, 386], [one night, 392], [Zurich, 38...","[Marlene Marder, Klaudia Schifferle, Lislot Ha...","[Zurich, Britain, London, LiLiPUT, Zurich]",[ ],"[Ain’t You,, Heidi’s Head, Eisiger Wind, Icy W...","[Rough Trade, Kleenex, Rough Trade’s, Swell Ma..."
4,22725,new start,taso,8.1,kevin lozano,1483660800000,electronic,2016,teklife,It is impossible to consider a given release b...,"[footwork, double, cup, rashad, way, start, tr...","[[footwork, 11], [way, 4], [producer, 3], [bes...","[[DJ Rashad’s, 380], [Double Cup, 388], [four ...","[DJ Rashad’s, Taso, Anastasios Ioannis Skalkos...","[San Francisco, Taso, Taso, Miami, UK, Taso]",[ ],"[Double Cup, Feelin, Pass That Shit, New Start...","[Taso, Gant-Man, Taso, Foodman]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18412,1535,let us replay!,coldcut,8.9,james p. wisdom,917308800000,"jazz,electronic",0,ninja tune,The marketing guys of yer average modern megac...,"[brand, equity, pale, new, money, product, goo...","[[brand, 7], [equity, 7], [pale, 2], [new, 2],...","[[these days, 391], [six\n months, 391], [C...","[Coldcut, Carl Craig, Virginia Epitome]",[ ],[ ],"[Atomic Moog 2000, Panopticon, More Beats + Pi...","[Coca-Cola, Pepsi, Coldcut, Timber, Irresistib..."
18413,1341,1999,cassius,4.8,james p. wisdom,917308800000,electronic,0,astralwerks,"Well, it's been two weeks now, and I guess it'...","[techno, sure, club, weeks, time, guys, dilige...","[[techno, 3], [sure, 3], [time, 1], [guys, 1],...","[[two weeks, 391], [Cassius, 380], ['70s, 391]...","[Cassius, flava, Astralwerks]",[France],[ ],"[La Monde, Foxy""\n , Club Soixante Quinze]",[ ]
18414,5376,out of tune,mojave 3,6.3,jason josephes,916099200000,rock,0,4ad,"Out of Tune is a Steve Martin album. Yes, I'l...","[martin, man, slowdive, steve, funny, laughter...","[[funny, 4], [laughter, 3], [tune, 2], [time, ...","[[Steve Martin, 380], [Steve Martin, 380], [Ra...","[Steve Martin, Steve Martin, Martin, Steve Mar...",[Seattle],"[Cellophane Square, University Avenue]","[Rambli

In [81]:
only_words = []
only_numbers = []

for i in range(len(key_words)):
    a = key_words[i]
    if not a:
        only_numbers.append([' '])
        only_words.append([' '])
        continue
    
            
    number = list(list(zip(*a))[1])
    words = list(list(zip(*a))[0])
    only_numbers.append(number)
    only_words.append(words)
    

    

In [24]:
only_organization = []
only_persons = []
only_places = []
only_work_of_art = []
only_facilities = []
only_events = []
only_norp = []
only_products = []
only_law = []
entities = list(data['entities'])

for i in range(len(entities)):
    a = entities[i]
    organization = []
    persons = []
    places = []
    work_of_art = []
    facilities = []
    events = []
    norp = []
    product = []
    law = []
    if not a:
        only_organization.append([' '])
        only_persons.append([' '])
        only_places.append([' '])
        only_work_of_art.append([' '])
        only_facilities.append([' '])
        only_events.append([' '])
        only_norp.append([' '])
        only_products.append([' '])
        only_law.append([' '])
        continue

    for ent in a:
        #if ent[1] == 391 or ent[1] == 397 or ent[1] == 396 or ent[1] == 392 or ent[1] == 394 or ent[1] == 395 or ent[1]==393:
         #   continue
        if ent[1] == 384 or ent[1] == 385:
            places.append(ent[0])
        elif ent[1] == 383:
            organization.append(ent[0])
        elif ent[1] == 380:
            persons.append(ent[0])
        elif ent[1] == 388:
            work_of_art.append(ent[0])
        elif ent[1] == 9191306739292312949:
            facilities.append(ent[0])
        elif ent[1] == 387:
            events.append(ent[0])
        elif ent[1] == 381 or ent[1] == 389:
            norp.append(ent[0])
        elif ent[1] == 386:
            product.append(ent[0])
        elif ent[1] == 390:
            law.append(ent[0])
        else:
            continue
            

    only_organization.append(organization)
    only_persons.append(persons)
    only_places.append(places)
    only_work_of_art.append(work_of_art)
    only_facilities.append(facilities)
    only_norp.append(norp)
    only_products.append(product)
    only_events.append(events)
    only_law.append(law)
    

In [23]:
print(only_others)

[[' '], [' '], [' '], [' '], [' '], [' '], [' '], [' '], [' '], [' '], [' '], [' '], [' '], [' ']]


In [25]:
data['persons'] = only_persons
data['places'] = only_places
data['facilities'] = only_facilities
data['organizations'] = only_organization
data['work_of_art'] = only_work_of_art
data['products'] = only_products
data['law'] = only_law
data['events'] = only_events
data['norp'] = only_norp

In [26]:
data

,reviewid,title,artist,score,author,review_publication_date,genre,song_release_year,record_label,review_content,...,entities,persons,places,facilities,work_of_art,organizations,products,law,events,norp
0,22703,mezzanine,massive attack,9.3,nate patrin,1483833600000,electronic,1998,virgin,"“Trip-hop” eventually became a ’90s punchline,...",...,"[[’90s, 391], [today, 391], [Bristol, 384], [m...","[Portishead, Geoff Barrow, Portishead, Robert ...","[Bristol, Jamaica, Bristol, Memphis, UK, Earth...",[Del Naja],"[The Conversation’s Gene Hackman, Inertia Cree...","[Pre-Millennium Tension, Massive Attack, Mezza...",[],[],[],[]
1,22721,prelapsarian,krallice,7.9,zoe camp,1483747200000,metal,2016,hathenter,"Eight years, five albums, and two EPs in, the ...",...,"[[Eight years, 391], [five, 397], [two, 397], ...","[Colin Marston, Mick Barr’s, Lev Weinstein’s, ...","[New York, Prelapsarian, Prelapsarian]",[the Towers of Terror],"[Transformation Chronicles, Lotus Throne, Hate...","[Krallice, Krallice, releases—2015, Hyperion, ...",[],[],[the Winter Solstice],[]
2,22659,all of them naturals,uranium club,7.3,david glickman,1483747200000,rock,2016,"static shock,fashionable idiots",Minneapolis’ Uranium Club seem to revel in bei...,...,"[[Minneapolis, 384], [Uranium Club, 383], [las...","[Devo, teddy]","[Minneapolis, Detroit]",[],"[Operation Pt, The Lottery, Opus, That Clown’s...","[Uranium Club, Human Exploration, Them Natural...",[],[],[],[British]
3,22661,first songs,"kleenex, liliput",9.0,jenn pelly,1483660800000,rock,2016,"kill rock stars,mississippi",Kleenex began with a crash. It transpired one ...,...,"[[Kleenex, 386], [one night, 392], [Zurich, 38...","[Marlene Marder, Klaudia Schifferle, Lislot Ha...","[Zurich, Britain, London, LiLiPUT, Zurich]",[],"[Ain’t You,, Heidi’s Head, Eisiger Wind, Icy W...","[Rough Trade, Kleenex, Rough Trade’s, Swell Ma...","[Kleenex, Kleenex, Lislot, Kleenex, Kleenex EP...",[then—“Beri-Beri],[],"[Swiss, Marxist, English, Swiss]"
4,22725,new start,taso,8.1,kevin lozano,1483660800000,electronic,2016,teklife,It is impossible to consider a given release b...,...,"[[DJ Rashad’s, 380], [Double Cup, 388], [four ...","[DJ Rashad’s, Taso, Anastasios Ioannis Skalkos...","[San Francisco, the Bay Area, Taso, Taso, Miam...",[],"[Double Cup, Feelin, Pass That Shit, New Start...","[Taso, Gant-Man, Taso, Foodman]","[Teklife, bpm]",[],[],[British]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18412,1535,let us replay!,coldcut,8.9,james p. wisdom,917308800000,"jazz,electronic",0,ninja tune,The marketing guys of yer average modern megac...,...,"[[these days, 391], [six\n months, 391], [C...","[Coldcut, Carl Craig, Virginia Epitome]",[],[],"[Atomic Moog 2000, Panopticon, More Beats + Pi...","[Coca-Cola, Pepsi, Coldcut, Timber, Irresistib...",[],[],[],[]
18413,1341,1999,cassius,4.8,james p. wisdom,917308800000,electronic,0,astralwerks,"Well, it's been two weeks now, and I guess it'...",...,"[[two weeks, 391], [Cassius, 380], ['70s, 391]...","[Cassius, flava, Astralwerks]",[France],[],"[La Monde, Foxy""\n , Club Soixante Quinze]",[],[],[],[],[]
18414,5376,out of tune,mojave 3,6.3,jason josephes,916099200000,rock,0,4ad,"Out of Tune is a Steve Martin album. Yes, I'l...",...,"[[Steve Martin, 380], [Steve Martin, 380], [Ra...","[Steve Martin, Steve Martin, Martin, Steve Mar...","[Seattle, Mojave]","[Cellophane Square, University Avenue]","[Ramblin' Man"" thang, The Jerk, The Man With T...","[Patch Adams, Martin, Mojave 3, Slowdive, Slow...",[],[],[],[Eurpoean]
18415,2413,"singles breaking up, vol. 1",don caballero,7.2,james p. wisdom,916099200000,"rock,metal,experimental",1999,touch and go,"Well, kids, I just went back and re-read my re...",...,"[[Don Caballero, 380], [NPR, 383], [This Ameri...","[Don Caballero, Don Caballero, Don C.]",[Singles],[],[],"[NPR, This American Life, jam-rok, Ketamine]",[],[],[],[]


In [27]:
for word in ['organizations', 'places', 'persons', 'facilities', 'law', 'work_of_art', 'events', 'products', 'norp']:
    keywords = list(data[word])
    new = [x if x else [' '] for x in keywords]
    data = data.drop(columns=word)
    data[word] = new
data

,reviewid,title,artist,score,author,review_publication_date,genre,song_release_year,record_label,review_content,...,entities,organizations,places,persons,facilities,law,work_of_art,events,products,norp
0,22703,mezzanine,massive attack,9.3,nate patrin,1483833600000,electronic,1998,virgin,"“Trip-hop” eventually became a ’90s punchline,...",...,"[[’90s, 391], [today, 391], [Bristol, 384], [m...","[Pre-Millennium Tension, Massive Attack, Mezza...","[Bristol, Jamaica, Bristol, Memphis, UK, Earth...","[Portishead, Geoff Barrow, Portishead, Robert ...",[Del Naja],[ ],"[The Conversation’s Gene Hackman, Inertia Cree...",[ ],[ ],[ ]
1,22721,prelapsarian,krallice,7.9,zoe camp,1483747200000,metal,2016,hathenter,"Eight years, five albums, and two EPs in, the ...",...,"[[Eight years, 391], [five, 397], [two, 397], ...","[Krallice, Krallice, releases—2015, Hyperion, ...","[New York, Prelapsarian, Prelapsarian]","[Colin Marston, Mick Barr’s, Lev Weinstein’s, ...",[the Towers of Terror],[ ],"[Transformation Chronicles, Lotus Throne, Hate...",[the Winter Solstice],[ ],[ ]
2,22659,all of them naturals,uranium club,7.3,david glickman,1483747200000,rock,2016,"static shock,fashionable idiots",Minneapolis’ Uranium Club seem to revel in bei...,...,"[[Minneapolis, 384], [Uranium Club, 383], [las...","[Uranium Club, Human Exploration, Them Natural...","[Minneapolis, Detroit]","[Devo, teddy]",[ ],[ ],"[Operation Pt, The Lottery, Opus, That Clown’s...",[ ],[ ],[British]
3,22661,first songs,"kleenex, liliput",9.0,jenn pelly,1483660800000,rock,2016,"kill rock stars,mississippi",Kleenex began with a crash. It transpired one ...,...,"[[Kleenex, 386], [one night, 392], [Zurich, 38...","[Rough Trade, Kleenex, Rough Trade’s, Swell Ma...","[Zurich, Britain, London, LiLiPUT, Zurich]","[Marlene Marder, Klaudia Schifferle, Lislot Ha...",[ ],[then—“Beri-Beri],"[Ain’t You,, Heidi’s Head, Eisiger Wind, Icy W...",[ ],"[Kleenex, Kleenex, Lislot, Kleenex, Kleenex EP...","[Swiss, Marxist, English, Swiss]"
4,22725,new start,taso,8.1,kevin lozano,1483660800000,electronic,2016,teklife,It is impossible to consider a given release b...,...,"[[DJ Rashad’s, 380], [Double Cup, 388], [four ...","[Taso, Gant-Man, Taso, Foodman]","[San Francisco, the Bay Area, Taso, Taso, Miam...","[DJ Rashad’s, Taso, Anastasios Ioannis Skalkos...",[ ],[ ],"[Double Cup, Feelin, Pass That Shit, New Start...",[ ],"[Teklife, bpm]",[British]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18412,1535,let us replay!,coldcut,8.9,james p. wisdom,917308800000,"jazz,electronic",0,ninja tune,The marketing guys of yer average modern megac...,...,"[[these days, 391], [six\n months, 391], [C...","[Coca-Cola, Pepsi, Coldcut, Timber, Irresistib...",[ ],"[Coldcut, Carl Craig, Virginia Epitome]",[ ],[ ],"[Atomic Moog 2000, Panopticon, More Beats + Pi...",[ ],[ ],[ ]
18413,1341,1999,cassius,4.8,james p. wisdom,917308800000,electronic,0,astralwerks,"Well, it's been two weeks now, and I guess it'...",...,"[[two weeks, 391], [Cassius, 380], ['70s, 391]...",[ ],[France],"[Cassius, flava, Astralwerks]",[ ],[ ],"[La Monde, Foxy""\n , Club Soixante Quinze]",[ ],[ ],[ ]
18414,5376,out of tune,mojave 3,6.3,jason josephes,916099200000,rock,0,4ad,"Out of Tune is a Steve Martin album. Yes, I'l...",...,"[[Steve Martin, 380], [Steve Martin, 380], [Ra...","[Patch Adams, Martin, Mojave 3, Slowdive, Slow...","[Seattle, Mojave]","[Steve Martin, Steve Martin, Martin, Steve Mar...","[Cellophane Square, University Avenue]",[ ],"[Ramblin' Man"" thang, The Jerk, The Man With T...",[ ],[ ],[Eurpoean]
18415,2413,"singles breaking up, vol. 1",don caballero,7.2,james p. wisdom,916099200000,"rock,metal,experimental",1999,touch and go,"Well, kids, I just went back and re-read my re...",...,"[[Don Caballero, 380], [NPR, 383], [This Ameri...","[NPR, This American Life, jam-rok, Ketamine]",[Singles],"[Don Caballero, Don Caballero, Don C.]",[ ],[ ],[ ],[ ],[ ],[ ]


In [28]:
data.to_json('reviews_with_entities_split_all.json', orient='records')

In [87]:
data = data.drop(columns=['keyword_frequency'])
data.to_json('reviews_with_only_keywords25.json', orient='records')

In [88]:
data

,reviewid,title,artist,score,author,review_publication_date,genre,song_release_year,record_label,review_content,keywords
0,22703,mezzanine,massive attack,9.3,nate patrin,1483833600000,electronic,1998,virgin,"“Trip-hop” eventually became a ’90s punchline,...","[late, del, naja, hop, band, 3d, andy, man, tr..."
1,22721,prelapsarian,krallice,7.9,zoe camp,1483747200000,metal,2016,hathenter,"Eight years, five albums, and two EPs in, the ...","[band, barr, metal, post, mcmaster, winter, de..."
2,22659,all of them naturals,uranium club,7.3,david glickman,1483747200000,rock,2016,"static shock,fashionable idiots",Minneapolis’ Uranium Club seem to revel in bei...,"[punk, band, human, exploration, ep, scene, na..."
3,22661,first songs,"kleenex, liliput",9.0,jenn pelly,1483660800000,rock,2016,"kill rock stars,mississippi",Kleenex began with a crash. It transpired one ...,"[punk, marder, scene, beri, early, chords, dru..."
4,22725,new start,taso,8.1,kevin lozano,1483660800000,electronic,2016,teklife,It is impossible to consider a given release b...,"[footwork, double, cup, rashad, way, start, tr..."
...,...,...,...,...,...,...,...,...,...,...,...
18412,1535,let us replay!,coldcut,8.9,james p. wisdom,917308800000,"jazz,electronic",0,ninja tune,The marketing guys of yer average modern megac...,"[brand, equity, pale, new, money, product, goo..."
18413,1341,1999,cassius,4.8,james p. wisdom,917308800000,electronic,0,astralwerks,"Well, it's been two weeks now, and I guess it'...","[techno, sure, club, weeks, time, guys, dilige..."
18414,5376,out of tune,mojave 3,6.3,jason josephes,916099200000,rock,0,4ad,"Out of Tune is a Steve Martin album. Yes, I'l...","[martin, man, slowdive, steve, funny, laughter..."
18415,2413,"singles breaking up, vol. 1",don caballero,7.2,james p. wisdom,916099200000,"rock,metal,experimental",1999,touch and go,"Well, kids, I just went back and re-read my re...","[sore-, jamming, review, sharp, drums, oppress..."
